In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

In [2]:
from langchain_core.prompts import PromptTemplate

demo_template = '''I want you to act as a acting financial advisor for people.
In an easy way, explain the basics of {financial_concept}.'''

prompt=PromptTemplate(
    input_variables=['financial_concept'],
    template= demo_template
)

prompt.format(financial_concept='income tax')

'I want you to act as a acting financial advisor for people.\nIn an easy way, explain the basics of income tax.'

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(temperature=0.7)

parser = StrOutputParser()

chain1 = prompt | llm | parser


In [5]:
chain1.invoke('GDP')

"Sure! GDP stands for Gross Domestic Product. It is a measure of the total value of all goods and services produced within a country's borders in a specific time period, usually annually or quarterly. \n\nGDP is an important indicator of a country's economic health and growth. It helps us understand how well the economy is performing and can give us insights into trends like inflation, unemployment, and overall economic stability.\n\nThere are three main ways to calculate GDP: \n1. Production approach: This measures the total value of all goods and services produced in the country.\n2. Income approach: This measures the total income earned by all individuals and businesses in the country.\n3. Expenditure approach: This measures the total amount spent on goods and services within the country, including consumption, investment, government spending, and net exports.\n\nOverall, GDP is a key metric used by policymakers, investors, and economists to assess the health of an economy and make 

In [6]:
# Language Translation
from langchain_core.prompts import PromptTemplate

template='''In an easy way translate the following sentence '{sentence}' into {target_language}'''
language_prompt = PromptTemplate(
    input_variables=["sentence",'target_language'],
    template=template,
)
language_prompt.format(sentence="How are you",target_language='hindi')

"In an easy way translate the following sentence 'How are you' into hindi"

In [16]:
chain2 =language_prompt |llm |parser
chain2.invoke({'sentence':"Hello, How are you", 'target_language':'hindi'})

'नमस्ते, आप कैसे हैं। (Namaste, aap kaise hain)'

In [17]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Step 1: Create a list of few-shot examples
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# Step 2: Create a template for formatting each example
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [18]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [19]:
print(few_shot_prompt.format(input='big'))

Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: big
Antonym: 


In [22]:
chain3 = few_shot_prompt | llm | parser

chain3.invoke({'input': "big"})



'small'